### Downloading the data

In [6]:
#imports

import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd

In [8]:
# Directory to save the data locally
%cd /Users/cicidu/MDS/m1_525

/Users/cicidu/MDS/m1_525


In [2]:
# Necessary metadata
article_id = 14096681  
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "figsharedailyrain/"

In [3]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)  
files = data["files"]             
files

[{'id': 26579150,
  'name': 'daily_rainfall_2014.png',
  'size': 58863,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579150',
  'supplied_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'computed_md5': 'fd32a2ffde300a31f8d63b1825d47e5e'},
 {'id': 26579171,
  'name': 'environment.yml',
  'size': 192,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579171',
  'supplied_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'computed_md5': '060b2020017eed93a1ee7dd8c65b2f34'},
 {'id': 26586554,
  'name': 'README.md',
  'size': 5422,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26586554',
  'supplied_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'computed_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c'},
 {'id': 26766812,
  'name': 'data.zip',
  'size': 814041183,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26766812',
  'supplied_md5': 'b517383f76e77bd03755a63a8f

In [9]:
files_to_dl = ["data.zip"]
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

In [10]:
with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(output_directory)

In [15]:
%ls -ltr figsharedailyrain

total 12109560
-rw-r--r--   1 cicidu  staff  814041183 29 Mar 14:53 data.zip
-rw-r--r--   1 cicidu  staff   95376895 29 Mar 14:53 MPI-ESM-1-2-HAM_daily_rainfall_NSW.csv
-rw-r--r--   1 cicidu  staff   94960113 29 Mar 14:53 AWI-ESM-1-1-LR_daily_rainfall_NSW.csv
-rw-r--r--   1 cicidu  staff   82474546 29 Mar 14:53 NorESM2-LM_daily_rainfall_NSW.csv
-rw-r--r--   1 cicidu  staff  127613760 29 Mar 14:53 ACCESS-CM2_daily_rainfall_NSW.csv
-rw-r--r--   1 cicidu  staff  232118894 29 Mar 14:53 FGOALS-f3-L_daily_rainfall_NSW.csv
-rw-r--r--   1 cicidu  staff  330360682 29 Mar 14:53 CMCC-CM2-HR4_daily_rainfall_NSW.csv
-rw-r--r--   1 cicidu  staff  254009247 29 Mar 14:53 MRI-ESM2-0_daily_rainfall_NSW.csv
-rw-r--r--   1 cicidu  staff  235661418 29 Mar 14:53 GFDL-CM4_daily_rainfall_NSW.csv
-rw-r--r--   1 cicidu  staff  294260911 29 Mar 14:53 BCC-CSM2-MR_daily_rainfall_NSW.csv
-rw-r--r--   1 cicidu  staff  295768615 29 Mar 14:53 EC-Earth3-Veg-LR_daily_rainfall_NSW.csv
-rw-r--r--   1 cicidu  staff  328852